In [1]:
import pandas as pd
from loguru import logger
from dotenv import load_dotenv
load_dotenv()

from src.generate import detect_bias
from src.utils import extract_response

In [34]:
splits = {'cognitive_bias': 'mbib-aggregated/cognitive-bias.csv', 'fake_news': 'mbib-aggregated/fake-news.csv', 'gender_bias': 'mbib-aggregated/gender-bias.csv', 'hate_speech': 'mbib-aggregated/hate-speech.csv', 'linguistic_bias': 'mbib-aggregated/linguistic-bias.csv', 'political_bias': 'mbib-aggregated/political-bias.csv', 'racial_bias': 'mbib-aggregated/racial-bias.csv', 'text_level_bias': 'mbib-aggregated/text-level-bias.csv'}
df = pd.read_csv("hf://datasets/mediabiasgroup/mbib-base/" + splits["racial_bias"])

In [35]:
for text in df[df['label'] == 0].sample(5)['text']:
    print(f'{text}\n')
    print('---')

best girl is now ultimate maid

---
if jews are so greedy why didnt the guards feed them sorry for this lol

---
polygamy is weird gay is cool and all but polygamy is ee

---
both or maybe worse you want to spread the idea that arabs are taught to hate jews and not the reality which is the other way around 

---
a rice cooker best ive ever spent my wife is from the philippines so we eat rice on a daily basis 

---


In [36]:
for text in df[df['label'] == 1].sample(5)['text']:
    print(f'{text}\n')
    print('---')

until i see it im just going to believe they are overreacting to something mild like muslims are da bomb

---
is it racist to say that though psychological studies show that black people are more typically more aggressive than other races 

---
one thing muslims got right is that the jews are indeed spawns of satan 

---
my wife is a nurse and family is full of doctors i have heard that often haha

---
omg squirmy and grubs strangers assume my girlfriend is my nurse was amazing i love you two and now i shall follow your dog as well 

---


## Generating balanced test and train sets

In [37]:
negative_examples = df[df['label'] == 0].sample(25)['text']
positive_examples = df[df['label'] == 1].sample(25)['text']

train_data = []
test_data = []

for text in negative_examples[0:20]:
    train_data.append({'text': text, 'label': 0})
for text in positive_examples[0:20]:
    train_data.append({'text': text, 'label': 1})
for text in negative_examples[20:25]:
    test_data.append({'text': text, 'label': 0})
for text in positive_examples[20:25]:
    test_data.append({'text': text, 'label': 1})

In [38]:
llm_providers = [
    "openai/gpt-4o-2024-08-06",
    "openai/gpt-4o-mini-2024-07-18",
    "anthropic/claude-3-5-sonnet-20241022",
    "google/gemini-1.5-pro-latest",
    "mistral/mistral-large-2407",
    "togetherai/DeepSeek-R1",
    "perplexity/sonar"
]

In [41]:
result = []
for provider in llm_providers:
    logger.info(f'Running provider {provider}')
    for input in train_data:
        logger.info(f'Running input {input["text"]} from train')
        try:
            response = detect_bias(input['text'], provider)
            logger.info(f'Got response, length: {len(response)}')
        except Exception as e:
            response = 'ERROR'
            logger.error(f'Error in detect_bias, {e}')
        result.append({'provider': provider, 'set': 'train', 'text': input['text'], 'label': input['label'], 'response': response})
    for input in test_data:
        logger.info(f'Running input {input["text"]} from test')
        try:
            response = detect_bias(input['text'], provider)
            logger.info(f'Got response, length: {len(response)}')
        except Exception as e:
            response = 'ERROR'
            logger.error(f'Error in detect_bias, {e}')
        result.append({'provider': provider, 'set': 'test', 'text': input['text'], 'label': input['label'], 'response': response})

2025-02-24 10:46:18.992 | INFO     | __main__:<module>:3 - Running provider openai/gpt-4o-2024-08-06
2025-02-24 10:46:18.993 | INFO     | __main__:<module>:5 - Running input focus on all crime not every black person is a aggressive criminal from train
2025-02-24 10:46:23.254 | INFO     | __main__:<module>:8 - Got response, length: 648
2025-02-24 10:46:23.255 | INFO     | __main__:<module>:5 - Running input the only drama here is from other redditors ops wife is only found to be poly by others op knew thats their thing  from train
2025-02-24 10:46:26.206 | INFO     | __main__:<module>:8 - Got response, length: 529
2025-02-24 10:46:26.206 | INFO     | __main__:<module>:5 - Running input because blacks are the problem not your crippling failure to be a functioning member of the world  from train
2025-02-24 10:46:29.639 | INFO     | __main__:<module>:8 - Got response, length: 522
2025-02-24 10:46:29.640 | INFO     | __main__:<module>:5 - Running input just because poor black people are ver

In [ ]:
result_df = pd.DataFrame(result)


In [52]:
result_df['predicted'] = result_df['response'].apply(extract_response)

In [55]:
result_df['score'] = result_df.apply(lambda x: 1 if x['label'] == x['predicted'] else 0, axis=1)

result_df.groupby(['provider', 'set']).agg({'score': 'sum'}).reset_index().pivot(index='provider', columns='set', values='score').reset_index().to_csv('data/scores.csv', index=False)

In [56]:
result_df.to_csv('data/results.csv', index=False)